<a href="https://colab.research.google.com/github/learningdollars/reddit-twitter-convo-data/blob/master/Reddit_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complete Reddit Selenum & BeautifulSoup Scraping



## Preparation

In [0]:
%%bash

chmod 777 /tmp
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing

In [2]:
# Install chromium, driver, and selenium
!pip install selenium
!pip install pandas
!pip install anytree
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Adding to path, unnecessary if files are moved to /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (78.0.3904.70-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 78 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


## Run

In [0]:
# Hyperparameters
PAGES = 3
HTTPS = "https://old.reddit.com"

In [0]:
# Set to headless
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as soup

import pandas as pd
import anytree
from anytree import Node, RenderTree
from pprint import pprint

options = webdriver.ChromeOptions()
options.add_argument("-no-sandbox")
options.add_argument("-headless")
options.add_argument("-disable-dev-shm-usage")

# Open it & a website, and get result
d = webdriver.Chrome("chromedriver", options=options)
d.get("https://old.reddit.com/r/GlobalOffensive/")
# print(d.page_source) # results

In [0]:
# !pip install pandas
import pandas as pd

In [40]:
# Get all posts
posts = []

for i in range(PAGES):
  wdposts = d.find_elements_by_class_name("thing")

  # Iterate through every post found
  for wdpost in wdposts:
    html = wdpost.get_attribute("outerHTML")
    attrs = soup(html, "html.parser").div.attrs
    posts.append(attrs)
  
  # Go to next page
  next = d.find_element_by_class_name("next-button").get_attribute("outerHTML")
  page = soup(next, "html.parser").a.attrs["href"]
  print(i, page)
  d.get(page)

# Create table using pandas DataFrame
dfposts = pd.DataFrame(posts)
dfposts

0 https://old.reddit.com/r/GlobalOffensive/?count=25&after=t3_dvfc43
1 https://old.reddit.com/r/GlobalOffensive/?count=50&after=t3_dvtmtt
2 https://old.reddit.com/r/GlobalOffensive/?count=75&after=t3_dvhry9


,class,id,onclick,data-fullname,data-type,data-gildings,data-whitelist-status,data-author,data-author-fullname,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-timestamp,data-url,data-permalink,data-domain,data-rank,data-comments-count,data-score,data-promoted,data-nsfw,data-spoiler,data-oc,data-num-crossposts,data-context,data-kind
0,"[, thing, id-t3_dvvs2e, linkflair, linkflair-d...",thing_t3_dvvs2e,click_thing(this),t3_dvvs2e,link,0,all_ads,CSGOMatchThreads,t2_mwknk,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573669152000,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,self.GlobalOffensive,,18,29,false,false,false,false,0,listing,NaN
1,"[, thing, id-t3_dvrfqa, linkflair, linkflair-g...",thing_t3_dvrfqa,click_thing(this),t3_dvrfqa,link,1,all_ads,YukonGoldRush,t2_3orh3c07,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573650120000,https://v.redd.it/qmu3xufsbgy31,/r/GlobalOffensive/comments/dvrfqa/my_a_bomb_s...,v.redd.it,1,240,6292,false,false,false,false,0,listing,gif
2,"[, thing, id-t3_dvx36i, linkflair, linkflair-g...",thing_t3_dvx36i,click_thing(this),t3_dvx36i,link,0,all_ads,banana_shavings,t2_kn55e,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573674282000,https://clips.twitch.tv/DifficultClumsyAlfalfa...,/r/GlobalOffensive/comments/dvx36i/rez_bonkers...,clips.twitch.tv,2,50,981,false,false,false,false,0,listing,NaN
3,"[, thing, id-t3_dvx2ec, linkflair, linkflair-g...",thing_t3_dvx2ec,click_thing(this),t3_dvx2ec,link,0,all_ads,naTh1i,t2_4gatuz1k,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573674201000,https://clips.twitch.tv/TsunderePunchyWoodcock...,/r/GlobalOffensive/comments/dvx2ec/twist_4k_hold/,clips.twitch.tv,3,40,550,false,false,false,false,0,listing,NaN
4,"[, thing, id-t3_dvuum1, linkflair, linkflair-g...",thing_t3_dvuum1,click_thing(this),t3_dvuum1,link,0,all_ads,Pepperinho,t2_16m6pj,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573665311000,https://i.redd.it/d1s09u8zkhy31.gif,/r/GlobalOffensive/comments/dvuum1/its_a_bird_...,i.redd.it,4,27,672,false,false,false,false,0,listing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,"[, thing, id-t3_dvzwlf, linkflair, linkflair-h...",thing_t3_dvzwlf,click_thing(this),t3_dvzwlf,link,0,all_ads,WhiteVegan,t2_oaw6oww,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573685509000,https://i.redd.it/rhq033hg9jy31.jpg,/r/GlobalOffensive/comments/dvzwlf/im_not_able...,i.redd.it,71,5,2,false,false,false,false,0,listing,NaN
72,"[, thing, id-t3_dvv2pk, linkflair, linkflair-f...",thing_t3_dvv2pk,click_thing(this),t3_dvv2pk,link,0,all_ads,unpopul_arse,t2_4vvwhqmb,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573666238000,https://clips.twitch.tv/GoodFlaccidDeerWoofer,/r/GlobalOffensive/comments/dvv2pk/the_human_s...,clips.twitch.tv,72,1,8,false,false,false,false,0,listing,NaN
73,"[, thing, id-t3_dvpqtt, odd, link, ]",thing_t3_dvpqtt,click_thing(this),t3_dvpqtt,link,0,all_ads,edgyguy1000,t2_35rd9pw7,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573639938000,https://i.redd.it/t7l8w56yhfy31.png,/r/GlobalOffensive/comments/dvpqtt/faceit_btw/,i.redd.it,73,9,30,false,false,false,false,0,listing,NaN
74,"[, thing, id-t3_dvzgkp, linkflair, linkflair-g...",thing_t3_dvzgkp,click_thing(this),t3_dvzgkp,link,0,all_ads,DrBubble1,t2_1cujhhd,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,1573683707000,https://clips.twitch.tv/WittyAgreeableCookieJo...,/r/GlobalOffensive/comments/dvzgkp/big_fail_no...,clips.twitch.tv,74,3,3,false,false,false,false,0,listing,NaN


In [0]:
dfposts["data-permalink"][0:1]

In [51]:
for postlink in dfposts["data-permalink"][0:1]:
  print("LINK:", postlink)
  d.get(HTTPS + postlink)
  print(d.page_source) # results

LINK: /r/GlobalOffensive/comments/dvvs2e/esl_pro_league_season_10_day_13_match_discussion/
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en" class=" js cssanimations csstransforms"><head><title>ESL Pro League Season 10 Day 13 Match Discussion : GlobalOffensive</title><meta name="keywords" content=" reddit, reddit.com, vote, comment, submit "><meta name="description" content="###Comments in this thread are sorted by new by default. If you want a live list of the newest comments check out the [Reddit Stream of this..."><meta name="referrer" content="always"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><link type="application/opensearchdescription+xml" rel="search" href="/static/opensearch.xml"><link rel="canonical" href="https://www.reddit.com/r/GlobalOffensive/comments/dvvs2e/esl_pro_league_season_10_day_13_match_discussion/"><link rel="amphtml" href="https://amp.reddit.com/r/GlobalOffensive/comments/dvvs2e/esl_pro_league_season_10_day_13_match_dis

In [54]:
# Get all comments from every posts
# for postlink in dfposts["data-permalink"]:
for postlink in dfposts["data-permalink"][0:1]:
  d.get(HTTPS + postlink)
  print(postlink)
  coms = []

  # Match all comments: div elements having both "thing" and "comment"
  # wdcoms = d.find_elements_by_css_selector("div")
  # Match all comments: div elements having "comment"
  wdcoms = d.find_elements_by_css_selector("div.comment")

  # Iterate through every comment found
  for wdcom in wdcoms:
    text = wdcom.find_element_by_css_selector("div.usertext-body").text
    try:
      vote = int((wdcom.find_element_by_css_selector("span.score.unvoted").text).split(' ')[0])
    except NoSuchElementException:
      vote = None

    html = wdcom.get_attribute("outerHTML")
    attrs = soup(html, "html.parser").div.attrs

    attrs.update({'comment':text, 'vote':vote})

    pprint(attrs)
    # print(vote, text)
    # print(attrs)

    coms.append(attrs)

dfcoms = pd.DataFrame(coms)
dfcoms

/r/GlobalOffensive/comments/dvvs2e/esl_pro_league_season_10_day_13_match_discussion/
{'class': ['',
           'thing',
           'id-t1_f7g08q7',
           'noncollapsed',
           'comment',
           'score-hidden',
           ''],
 'comment': 'North winning 1/6 pistols I believe?\n'
            "Absolutely terrible stat to boast if you're hoping to win ANY "
            'bo3.',
 'data-author': 'thorizzle117',
 'data-author-fullname': 't2_4u8bl',
 'data-fullname': 't1_f7g08q7',
 'data-gildings': '0',
 'data-permalink': '/r/GlobalOffensive/comments/dvvs2e/esl_pro_league_season_10_day_13_match_discussion/f7g08q7/',
 'data-replies': '0',
 'data-subreddit': 'GlobalOffensive',
 'data-subreddit-fullname': 't5_2sqho',
 'data-subreddit-prefixed': 'r/GlobalOffensive',
 'data-subreddit-type': 'public',
 'data-type': 'comment',
 'id': 'thing_t1_f7g08q7',
 'onclick': 'click_thing(this)',
 'vote': None}
{'class': ['', 'thing', 'id-t1_f7ff5kt', 'noncollapsed', 'comment', ''],
 'comment': '"A

,class,id,onclick,data-fullname,data-type,data-gildings,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-author,data-author-fullname,data-replies,data-permalink,comment,vote
0,"[, thing, id-t1_f7g08q7, noncollapsed, comment...",thing_t1_f7g08q7,click_thing(this),t1_f7g08q7,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,thorizzle117,t2_4u8bl,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,North winning 1/6 pistols I believe?\nAbsolute...,NaN
1,"[, thing, id-t1_f7ff5kt, noncollapsed, comment, ]",thing_t1_f7ff5kt,click_thing(this),t1_f7ff5kt,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,Impulseps,t2_770cu,1,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,"""And it's twist again, like it was last summer...",4.0
2,"[, thing, id-t1_f7flyc2, noncollapsed, comment, ]",thing_t1_f7flyc2,click_thing(this),t1_f7flyc2,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,MorroWtje,t2_n7xs0,1,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,Chubby Checker innit?,1.0
3,"[, thing, id-t1_f7flz2w, noncollapsed, comment, ]",thing_t1_f7flz2w,click_thing(this),t1_f7flz2w,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,Impulseps,t2_770cu,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,ye,1.0
4,"[, thing, id-t1_f7fckbh, noncollapsed, comment, ]",thing_t1_f7fckbh,click_thing(this),t1_f7fckbh,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,prudentio,t2_4an0u9ug,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,Rez wtf are you doing,1.0
5,"[, thing, id-t1_f7fb8ib, noncollapsed, comment, ]",thing_t1_f7fb8ib,click_thing(this),t1_f7fb8ib,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,abaac,t2_evwn8,1,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,"wow, Harry and Hugo are quickly becoming my fa...",3.0
6,"[, thing, id-t1_f7fct5w, noncollapsed, comment, ]",thing_t1_f7fct5w,click_thing(this),t1_f7fct5w,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,_c0ldburN_,t2_7usb11,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,Yeah great duo and banter,1.0
7,"[, thing, id-t1_f7fa7j6, noncollapsed, comment, ]",thing_t1_f7fa7j6,click_thing(this),t1_f7fa7j6,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,Spork_Revolution,t2_x3w85,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,"You cannot spell Mirage without ""I rage"" - Jus...",3.0
8,"[, thing, id-t1_f7fa1sj, noncollapsed, comment, ]",thing_t1_f7fa1sj,click_thing(this),t1_f7fa1sj,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,hesbackforgood,t2_50fzilgl,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,So I’ve been taking a bit of a break from watc...,2.0
9,"[, thing, id-t1_f7f9zhy, noncollapsed, comment, ]",thing_t1_f7f9zhy,click_thing(this),t1_f7f9zhy,comment,0,GlobalOffensive,r/GlobalOffensive,t5_2sqho,public,xxcaymxx,t2_29b2hx53,0,/r/GlobalOffensive/comments/dvvs2e/esl_pro_lea...,Damm this matchup is so good,1.0


In [35]:
print(dfcoms)

                                                class  ...      vote
0   [, thing, id-t1_f7ff5kt, noncollapsed, comment, ]  ...  4 points
1   [, thing, id-t1_f7flyc2, noncollapsed, comment, ]  ...   1 point
2   [, thing, id-t1_f7flz2w, noncollapsed, comment, ]  ...   1 point
3   [, thing, id-t1_f7fckbh, noncollapsed, comment, ]  ...   1 point
4   [, thing, id-t1_f7fb8ib, noncollapsed, comment, ]  ...  3 points
5   [, thing, id-t1_f7fct5w, noncollapsed, comment, ]  ...   1 point
6   [, thing, id-t1_f7fa7j6, noncollapsed, comment, ]  ...  3 points
7   [, thing, id-t1_f7fa1sj, noncollapsed, comment, ]  ...  2 points
8   [, thing, id-t1_f7f9zhy, noncollapsed, comment, ]  ...   1 point
9   [, thing, id-t1_f7f8o4h, noncollapsed, comment, ]  ...   1 point
10  [, thing, id-t1_f7f8h4p, noncollapsed, comment, ]  ...   1 point
11  [, thing, id-t1_f7f8gwx, noncollapsed, comment, ]  ...   1 point
12  [, thing, id-t1_f7f8b3p, noncollapsed, comment, ]  ...  2 points
13  [, thing, id-t1_f7f85ga, nonco